As always, start by import `datajoint` library.

In [ ]:
import datajoint as dj

# Create schema and tables, and insert data

We first create a schema to define tables in. Be sure to replace `'YOUR_USERNAME'` with your Database username (e.g. if your user name is `john`, make it `john_calcium`)

In [ ]:
schema = dj.schema('YOUR_USERNAME_calcium')

We now create a table for Mouse, by defining a python class `Mouse`

In [ ]:
@schema
class Mouse(dj.Manual):
    definition = """
    # Experimental animals
    mouse_id             : int                          # Unique animal ID
    ---
    dob=null             : date                         # date of birth
    sex="unknown"        : enum('M','F','unknown')      # sex
    mouse_notes=""       : varchar(4096)                # other comments and distinguishing features
    """

`definition` is a property of the class which provides the information for DataJoint to create a table in the database. Above the `---` is the primary key of the table, in this case, `mouse_id` that uniquely identify a mouse. Below the `---` are the secondary attributes of the table, in this case, some other facts about the mouse.

Now let's insert a few mice into the newly defined table. You can either insert 1 at a time as a dictionary ...

In [ ]:
Mouse.insert1(
{'dob': '2016-01-05',
 'mouse_id': 8602,
 'mouse_notes': '',
 'sex': 'M'})
Mouse.insert1({'dob': '2016-01-06',
 'mouse_id': 8603,
 'mouse_notes': '',
 'sex': 'M'})

or many together in a batch, as a list of dictionaries.

In [ ]:
Mouse.insert([
    {'dob': "2016-01-04", 'mouse_id': 8623, 'sex': 'M'},
    {'dob': "2016-01-22", 'mouse_id': 8804, 'sex': 'F'}])

Similarly, let's now define a table for scan sessions.

In [ ]:
@schema
class Session(dj.Manual):
    definition = """
    -> Mouse
    session              : smallint                     # session number
    ---
    session_date         : date                         # date
    person               : varchar(100)                 # researcher name
    scan_path            : varchar(255)                 # file path for TIFF stacks
    """

The `->` indicates that the current table inherits the primary key of table `Mouse`, as a part of it's own primary key. Other than the `mouse_id`, there is also a `session` that identify different sessions of one mouse.

We can look at tables and their relationships by displaying the entitiy relationship diagram (ERD).

In [ ]:
dj.Di(schema)

Let's try inserting a few sessions manually.

In [ ]:
Session.insert1(
    {'mouse_id': 8623, 
     'session': 1, 
     'session_date': "2016-02-17",
     'person': 'Jake', 
     'scan_path': '/scratch01/Two-Photon/160217'
    })

Session.insert1(
    {   'mouse_id': 8804,
        'person': 'Manolis',
        'scan_path': '/scratch03/Two-Photon/Jake/160302',
        'session': 1,
        'session_date': "2016-03-02"})

# Simple queries

We can now perform some simple queries from these tables.

View all entries in a table:

In [ ]:
Mouse()

## Restrictions `&` and `-`: filtering data with certain conditions

In [ ]:
# male mice
Mouse & 'sex="M"'

In [ ]:
# male mice, restrict with a dict
Mouse & {'sex': 'M'}

In [ ]:
# mice born after a certain date
Mouse & 'dob > "2016-01-20"'

In [ ]:
# mice born within a time range
Mouse & 'dob between "2016-01-05" and "2016-01-10"'

DataJoint supports most of the MySQL restrictions, for more, check documentation of MySQL.

In [ ]:
# combination of different restrictors
Mouse & 'dob between "2016-01-05" and "2016-01-10"' & 'sex = "M"'

In [ ]:
# restriction with other tables, e.gl. all mice for which one or more sessions exist
Mouse & Session

In [ ]:
# negative restriction, mouse that is not 8623
Mouse - {'mouse_id': 8623}

In [ ]:
# mouse that does not have a session
Mouse - Session

## Join `*`: gather information from different tables

In [ ]:
Mouse * Session

The outcome of this query only takes the common entries (share primary key) of the two tables, in this case, only the mice that have a session, and then list the other fields.

## Projection `.proj()`: focus on attributes of interest

In [ ]:
Mouse.proj()

In [ ]:
Mouse.proj('dob', 'sex')

### Rename attribute with `proj()`

In [ ]:
Mouse.proj(birth_date='dob')

### Perform simple computations with `proj`

Example: compute the age of animal when performing each session

In [ ]:
# First get the date of birth and the session date into the same query with join
q = Mouse * Session

In [ ]:
q

In [ ]:
# Then compute the age with proj()
q.proj(age='session_date - dob')

## Aggregation `.aggr()`: simple computation of one table against another table

Example: how many sessions does each mouse do so far?

In [ ]:
Mouse.aggr(Session, n='count(*)')

# Fetch data: `fetch()`

So far, we have only performed queries, which just return an overview of results. To actually get data in python formats, we can use the `fetch()` method. This method could be used on all query results.

In [ ]:
# fetch all entries and all fields as numpy structured array
Mouse.fetch()

In [ ]:
# as a list of dictionaries
Mouse.fetch(as_dict=True)

In [ ]:
# as pandas dataframe
Mouse.fetch(format='frame')

In [ ]:
# Fetch a subset of fields
dob, sex = Mouse.fetch('dob', 'sex')

In [ ]:
dob

In [ ]:
sex

In [ ]:
# only fetch the primary key
Mouse.fetch('KEY')

In [ ]:
# Fetch 1 entry with fetch1()
(Mouse & 'mouse_id=8602').fetch1('dob')

In [ ]:
# fetch() returns an array even if only one entry is fetched.
(Mouse & 'mouse_id=8602').fetch('dob')

# Manipulate tables

In [ ]:
# delete entries
(Mouse & {'mouse_id': 8623}).delete()

In [ ]:
# drop a table
Mouse.drop()